In [32]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin.gz', binary=True, limit=1000000) 
wv.save_word2vec_format('vectors.csv')

In [33]:
import pandas as pd

phrases_df = pd.read_csv('phrases.csv', encoding= 'unicode_escape')
phrases_df.head()

,Phrases
0,how company compares to its peers?
1,what is the detailed income statement breakdow...
2,world premium penetration in 2020
3,How does the forecasted insurance premium pene...
4,what are the total losses for companies in cou...


In [34]:
import numpy as np

def get_phrase_vector(phrase):
    words = phrase.split()
    vector_sum = np.zeros(wv.vector_size)
    for word in words:
        if word in wv:
            vector_sum += wv[word]
    return vector_sum / len(words)

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = []
for i in range(len(phrases_df)):
    row = []
    for j in range(len(phrases_df)):
        vector1 = get_phrase_vector(phrases_df.iloc[i]['Phrases'])
        vector2 = get_phrase_vector(phrases_df.iloc[j]['Phrases'])
        similarity = cosine_similarity([vector1], [vector2])
        row.append(similarity[0][0])
    similarity_matrix.append(row)

In [36]:
def find_closest_match(user_input, phrases_df):
    user_vector = get_phrase_vector(user_input)
    similarities = []
    for phrase in phrases_df['Phrases']:
        phrase_vector = get_phrase_vector(phrase)
        similarity = cosine_similarity([user_vector], [phrase_vector])[0][0]
        similarities.append(similarity)
    closest_index = np.argmax(similarities)
    closest_phrase = phrases_df.iloc[closest_index]['Phrases']
    distance = 1 - similarities[closest_index]  
    return closest_phrase, distance


In [37]:
input_phrase = input('Enter a phrase: ')
closest_phrase, distance = find_closest_match(input_phrase, phrases_df)
print('Closest match: ', closest_phrase)
print('Distance to closest match: ', distance)

Closest match:  How many passengers are carried in ?
Distance to closest match:  0.1782007280129282
